<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/dacon_timeseries/blob/main/final_model/%EC%8B%9C%EA%B3%84%EC%97%B4_1%ED%8C%80_private_inference_%EC%9D%BC%EC%82%AC%EC%9D%BC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test 데이터 전처리 후 inference
=> ulsan_weather_06, dangjin_weather_06 최신 업데이트 버전으로 가져오기

* 이미지 한 장이 하루여야 함
* 72시간의 데이터를 갖고 73번째 시간대의 데이터를 예측하는것

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Mounted at ./gdrive


In [ ]:
dangjin_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/dangjin_weather_06.csv", encoding = 'utf-8')
ulsan_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/ulsan_weather_06.csv", encoding = 'utf-8')
#dangjin_sun = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/dangjin0702_0703_sunlight.csv", encoding = 'utf-8')
#ulsan_sun = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/ulsan0702_0703_sunlight.csv", encoding = 'utf-8')

In [ ]:
dangjin_weather_06.shape

(215, 14)

In [ ]:
def interpolatee(dangjin_fcst):
  dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate()  #interpolate() 구간 따로 안정하고 column 단위로 통째로 적용하면 되는거 맞음
  #dangjin_fcst['PrecipitationForm'] = dangjin_fcst['PrecipitationForm'].interpolate()
  #dangjin_fcst['PrecipitationProb'] = dangjin_fcst['PrecipitationProb'].interpolate()
  dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
  dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
  dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
  #dangjin_fcst['Precipitation'] = dangjin_fcst['Precipitation'].interpolate()
  #dangjin_fcst['Snow'] = dangjin_fcst['Snow'].interpolate()
  return dangjin_fcst

In [ ]:
dangjin_weather_06 = interpolatee(dangjin_weather_06)
ulsan_weather_06 = interpolatee(ulsan_weather_06)

In [ ]:
#Wind Direction 항목 angle에서 방향표시로 바꿔주기
def angle_to_dir(x):
    if pd.isnull(x):
        return np.nan
    elif x >= 22.5 and x <67.5:
        return 'NE'
    elif x >= 67.5 and x < 112.5:
        return 'E'
    elif x >= 112.5 and x < 157.5:
        return 'SE'
    elif x >= 157.5 and x < 202.5:
        return 'S'
    elif x >= 202.5 and x < 247.5:
        return 'WS'
    elif x >= 247.5 and x < 292.5:
        return 'W'
    elif x >= 292.5 and x < 337.5:
        return 'NW'
    elif x >= 337.5 or x < 22.5:
        return 'N'

def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))


In [ ]:
dangjin_weather_06['WindDirection_x'] = dangjin_weather_06['WindDirection'].apply(angle_to_cos)  
dangjin_weather_06['WindDirection_y'] = dangjin_weather_06['WindDirection'].apply(angle_to_sin)
dangjin_weather_06['Wind_x'] = dangjin_weather_06['WindDirection_x'] * dangjin_weather_06['WindSpeed']
dangjin_weather_06['Wind_y'] = dangjin_weather_06['WindDirection_y'] * dangjin_weather_06['WindSpeed']

ulsan_weather_06['WindDirection_x'] = ulsan_weather_06['WindDirection'].apply(angle_to_cos)
ulsan_weather_06['WindDirection_y'] = ulsan_weather_06['WindDirection'].apply(angle_to_sin)
ulsan_weather_06['Wind_x'] = ulsan_weather_06['WindDirection_x'] * ulsan_weather_06['WindSpeed']
ulsan_weather_06['Wind_y'] = ulsan_weather_06['WindDirection_y'] * ulsan_weather_06['WindSpeed']

dangjin_weather_06['WindDirection'] = dangjin_weather_06['WindDirection'].interpolate()
ulsan_weather_06['WindDirection'] = ulsan_weather_06['WindDirection'].interpolate()

#dangjin_weather_06['WindDirection_catg'] = dangjin_weather_06['WindDirection'].apply(angle_to_dir)
#ulsan_weather_06['WindDirection_catg'] = ulsan_weather_06['WindDirection'].apply(angle_to_dir)

In [ ]:
dangjin_weather_06['Wind_x'] = dangjin_weather_06['Wind_x'].interpolate()
dangjin_weather_06['Wind_y'] = dangjin_weather_06['Wind_y'].interpolate()

ulsan_weather_06['Wind_x'] = ulsan_weather_06['Wind_x'].interpolate()
ulsan_weather_06['Wind_y'] = ulsan_weather_06['Wind_y'].interpolate()

In [ ]:
dangjin_weather_06['hour'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.hour
dangjin_weather_06['day'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.day
dangjin_weather_06['month'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.month

ulsan_weather_06['hour'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.hour
ulsan_weather_06['day'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.day
ulsan_weather_06['month'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.month

In [ ]:
#이슬점 직접 계산해서 추가
b = 17.62
c = 243.12
gamma = (b * dangjin_weather_06['Temperature'] /(c + dangjin_weather_06['Temperature'])) + np.log(dangjin_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_weather_06['DewPoint'] = dewpoint  

gamma = (b *ulsan_weather_06['Temperature'] /(c + ulsan_weather_06['Temperature'])) + np.log(ulsan_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_weather_06['DewPoint'] = dewpoint

In [ ]:
dangjin_weather_06['방위각'] = dangjin_weather_06['방위각'].apply(lambda x : int(x[:3]))
dangjin_weather_06['적경'] = dangjin_weather_06['적경'].apply(lambda x : int(x.split(" ")[0]))
dangjin_weather_06['고도'] = dangjin_weather_06['고도'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_weather_06['적위'] = dangjin_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_weather_06['방위각'] = ulsan_weather_06['방위각'].apply(lambda x : int(x[:3]))
ulsan_weather_06['고도'] = ulsan_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_weather_06['적경'] = ulsan_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_weather_06['적위'] = ulsan_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [ ]:
using_col = ['Temperature', 'DewPoint', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각',	'고도',	'적경',	'적위','일조','일사']

dangjin_weather_06 = dangjin_weather_06[using_col].fillna(0)
ulsan_weather_06 = ulsan_weather_06[using_col].fillna(0)

In [ ]:
# Shift => period만큼 데이터 옮겨서 concat
# 해당 시간대에 대해 당일 예보, 하루 전 예보, 이틀 전 예보 이렇게 column을 늘려줌
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): #period부터 거꾸로 1칸씩 (71부터 0까지 출력)
    tmp = df.shift(i) #i는 period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,20.700000,18.820099,89.000000,9.0,1.710101e-01,-0.469846,1,6,6,-29.2554,6,23.1639,0.000000,0.000000
1,20.700000,18.820099,89.000000,9.0,6.577848e-01,0.239414,2,6,21,-26.3545,6,23.1632,0.000000,0.000000
2,20.900000,19.017257,89.000000,10.0,1.836970e-17,0.300000,3,6,35,-20.5601,6,23.1625,0.000000,0.000000
3,20.500000,18.801752,90.000000,9.0,2.394141e-01,0.657785,4,6,46,-13.0448,6,23.1619,0.000000,0.000000
4,20.500000,18.259954,87.000000,10.0,6.894400e-01,0.578509,5,6,57,-3.3908,6,23.1612,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,25.333333,23.262458,88.333333,4.0,-1.425936e+00,-4.303233,19,7,291,9.2453,7,22.3653,0.351233,0.044595
211,24.666667,23.217990,91.666667,4.0,-1.848042e+00,-3.845671,20,7,299,-1.2229,7,22.3636,0.000000,0.000000
212,24.000000,23.148352,95.000000,4.0,-2.178556e+00,-3.354682,21,7,309,-11.1143,7,22.3620,0.000000,0.000000
213,24.000000,23.148352,95.000000,4.0,-2.178556e+00,-3.354682,22,7,321,-19.3532,7,22.3604,0.000000,0.000000


In [ ]:
dangjin_weather_06 = shift(dangjin_weather_06, 71) 
ulsan_weather_06 = shift(ulsan_weather_06, 71) 

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.700000,18.820099,89.000000,9.0,1.710101e-01,-0.469846,1,6,6,-29.2554,6,23.1639,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.700000,18.820099,89.000000,9.0,1.710101e-01,-0.469846,1.0,6.0,6.0,-29.2554,6.0,23.1639,0.000000,0.000000,20.700000,18.820099,89.000000,9.0,6.577848e-01,0.239414,2,6,21,-26.3545,6,23.1632,0.000000,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.000000,9.0,1.710101e-01,-0.469846,1.0,6.0,6.0,-29.2554,6.0,23.1639,0.000000,0.000000,20.700000,18.820099,89.000000,9.0,6.577848e-01,0.239414,2.0,6.0,21.0,-26.3545,6.0,23.1632,0.000000,0.000000,20.900000,19.017257,89.000000,10.0,1.836970e-17,0.300000,3,6,35,-20.5601,6,23.1625,0.000000,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.000000,9.0,6.577848e-01,0.239414,2.0,6.0,21.0,-26.3545,6.0,23.1632,0.000000,0.000000,20.900000,19.017257,89.000000,10.0,1.836970e-17,0.300000,3.0,6.0,35.0,-20.5601,6.0,23.1625,0.000000,0.000000,20.500000,18.801752,90.000000,9.0,2.394141e-01,0.657785,4,6,46,-13.0448,6,23.1619,0.000000,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.000000,10.0,1.836970e-17,0.300000,3.0,6.0,35.0,-20.5601,6.0,23.1625,0.000000,0.000000,20.500000,18.801752,90.000000,9.0,2.394141e-01,0.657785,4.0,6.0,46.0,-13.0448,6.0,23.1619,0.000000,0.000000,20.500000,18.259954,87.000000,10.0,6.894400e-01,0.578509,5,6,57,-3.3908,6,23.1612,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,21.4,21.070422,98.0,0.0,9.958578e-01,8.356239e-01,20.0,7.0,300.0,-1.1624,6.0,22.5345,0.0,0.0,22.2,20.479850,90.0,0.0,3.500000e+00,4.286264e-16,21.0,7.0,310.0,-11.0332,6.0,22.5332,0.0,0.0,21.6,20.933888,96.0,0.0,2.800000e+00,3.429011e-16,22.0,7.0,321.0,-19.2450,6.0,22.5319,...,81.666667,4.0,-1.456679e+00,-4.151877,17.0,7.0,275.0,32.3904,7.0,22.3725,0.592201,0.231742,26.000000,23.280070,85.000000,4.0,-9.158832e-01,-4.711810,18.0,7.0,283.0,20.5029,7.0,22.3709,0.387920,0.125105,25.333333,23.262458,88.333333,4.0,-1.425936e+00,-4.303233,19,7,291,9.2453,7,22.3653,0.351233,0.044595
211,22.2,20.479850,90.0,0.0,3.500000e+00,4.286264e-16,21.0,7.0,310.0,-11.0332,6.0,22.5332,0.0,0.0,21.6,20.933888,96.0,0.0,2.800000e+00,3.429011e-16,22.0,7.0,321.0,-19.2450,6.0,22.5319,0.0,0.0,22.5,22.167677,98.0,0.0,1.409539e+00,-5.130302e-01,23.0,7.0,334.0,-25.4439,6.0,22.5305,...,85.000000,4.0,-9.158832e-01,-4.711810,18.0,7.0,283.0,20.5029,7.0,22.3709,0.387920,0.125105,25.333333,23.262458,88.333333,4.0,-1.425936e+00,-4.303233,19.0,7.0,291.0,9

In [ ]:
#앞에 72개는 nan 제거
'''
dangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]
ulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]
'''

'\ndangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]\nulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]\n'

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.700000,18.820099,89.000000,9.0,1.710101e-01,-0.469846,1,6,6,-29.2554,6,23.1639,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.700000,18.820099,89.000000,9.0,1.710101e-01,-0.469846,1.0,6.0,6.0,-29.2554,6.0,23.1639,0.000000,0.000000,20.700000,18.820099,89.000000,9.0,6.577848e-01,0.239414,2,6,21,-26.3545,6,23.1632,0.000000,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.000000,9.0,1.710101e-01,-0.469846,1.0,6.0,6.0,-29.2554,6.0,23.1639,0.000000,0.000000,20.700000,18.820099,89.000000,9.0,6.577848e-01,0.239414,2.0,6.0,21.0,-26.3545,6.0,23.1632,0.000000,0.000000,20.900000,19.017257,89.000000,10.0,1.836970e-17,0.300000,3,6,35,-20.5601,6,23.1625,0.000000,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.000000,9.0,6.577848e-01,0.239414,2.0,6.0,21.0,-26.3545,6.0,23.1632,0.000000,0.000000,20.900000,19.017257,89.000000,10.0,1.836970e-17,0.300000,3.0,6.0,35.0,-20.5601,6.0,23.1625,0.000000,0.000000,20.500000,18.801752,90.000000,9.0,2.394141e-01,0.657785,4,6,46,-13.0448,6,23.1619,0.000000,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.000000,10.0,1.836970e-17,0.300000,3.0,6.0,35.0,-20.5601,6.0,23.1625,0.000000,0.000000,20.500000,18.801752,90.000000,9.0,2.394141e-01,0.657785,4.0,6.0,46.0,-13.0448,6.0,23.1619,0.000000,0.000000,20.500000,18.259954,87.000000,10.0,6.894400e-01,0.578509,5,6,57,-3.3908,6,23.1612,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,21.4,21.070422,98.0,0.0,9.958578e-01,8.356239e-01,20.0,7.0,300.0,-1.1624,6.0,22.5345,0.0,0.0,22.2,20.479850,90.0,0.0,3.500000e+00,4.286264e-16,21.0,7.0,310.0,-11.0332,6.0,22.5332,0.0,0.0,21.6,20.933888,96.0,0.0,2.800000e+00,3.429011e-16,22.0,7.0,321.0,-19.2450,6.0,22.5319,...,81.666667,4.0,-1.456679e+00,-4.151877,17.0,7.0,275.0,32.3904,7.0,22.3725,0.592201,0.231742,26.000000,23.280070,85.000000,4.0,-9.158832e-01,-4.711810,18.0,7.0,283.0,20.5029,7.0,22.3709,0.387920,0.125105,25.333333,23.262458,88.333333,4.0,-1.425936e+00,-4.303233,19,7,291,9.2453,7,22.3653,0.351233,0.044595
211,22.2,20.479850,90.0,0.0,3.500000e+00,4.286264e-16,21.0,7.0,310.0,-11.0332,6.0,22.5332,0.0,0.0,21.6,20.933888,96.0,0.0,2.800000e+00,3.429011e-16,22.0,7.0,321.0,-19.2450,6.0,22.5319,0.0,0.0,22.5,22.167677,98.0,0.0,1.409539e+00,-5.130302e-01,23.0,7.0,334.0,-25.4439,6.0,22.5305,...,85.000000,4.0,-9.158832e-01,-4.711810,18.0,7.0,283.0,20.5029,7.0,22.3709,0.387920,0.125105,25.333333,23.262458,88.333333,4.0,-1.425936e+00,-4.303233,19.0,7.0,291.0,9

In [ ]:
#min max scaling
dangjin_min = dangjin_weather_06.min()
dangjin_max = dangjin_weather_06.max()

dangjin_weather_06 = (dangjin_weather_06 - dangjin_min) / (dangjin_max - dangjin_min)

In [ ]:
ulsan_min = ulsan_weather_06.min()
ulsan_max = ulsan_weather_06.max()

ulsan_weather_06 = (ulsan_weather_06 - ulsan_min) / (ulsan_max - ulsan_min)

# 2D Conv 모델

In [ ]:
import tensorflow as tf

In [ ]:
dangjin_weather_06.shape

(215, 1008)

In [ ]:
dangjin_weather_06 = dangjin_weather_06.values.reshape((dangjin_weather_06.shape[0], 72, 14, 1)) #(193, 864) -> (193, 72, 12, 1)
ulsan_weather_06 = ulsan_weather_06.values.reshape((ulsan_weather_06.shape[0], 72, 14, 1))

In [ ]:
dangjin_weather_06.shape

(215, 72, 14, 1)

In [ ]:
'''
dangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) 
ulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))
'''

'\ndangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) \nulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))\n'

In [ ]:
dangjin_weather_06 = np.squeeze(dangjin_weather_06, axis = -1) 
dangjin_weather_06 = np.stack((dangjin_weather_06,)*3, axis = -1)  

ulsan_weather_06 = np.squeeze(ulsan_weather_06, axis = -1)
ulsan_weather_06 = np.stack((ulsan_weather_06,)*3, axis = -1)

In [ ]:
ulsan_weather_06.shape

(215, 72, 14, 3)

In [ ]:
dangjin_test = dangjin_weather_06[-24:]
ulsan_test = ulsan_weather_06[-24:]

In [ ]:
dangjin_test.shape

(24, 72, 14, 3)

* Inference

In [ ]:
floating_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold4.h5")

In [ ]:
floating0 = floating_model0.predict(dangjin_test)
floating1 = floating_model1.predict(dangjin_test)
floating2 = floating_model2.predict(dangjin_test)
floating3 = floating_model3.predict(dangjin_test)
floating4 = floating_model4.predict(dangjin_test)

In [ ]:
floating2

array([[  1.0006521 ],
       [  0.97973347],
       [  0.96403456],
       [  1.1349999 ],
       [  1.3837398 ],
       [  1.5934992 ],
       [  1.8501136 ],
       [  2.322669  ],
       [  2.8086393 ],
       [  6.356037  ],
       [ 28.483809  ],
       [ 53.060886  ],
       [128.60178   ],
       [188.0825    ],
       [198.61687   ],
       [195.04378   ],
       [ 88.76227   ],
       [ 39.00084   ],
       [ 13.961651  ],
       [  3.3565326 ],
       [  1.3302926 ],
       [  1.3926677 ],
       [  1.4703648 ],
       [  1.6248199 ]], dtype=float32)

In [ ]:
warehouse_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold4.h5")

In [ ]:
warehouse0 = warehouse_model0.predict(dangjin_test)
warehouse1 = warehouse_model1.predict(dangjin_test)
warehouse2 = warehouse_model2.predict(dangjin_test)
warehouse3 = warehouse_model3.predict(dangjin_test)
warehouse4 = warehouse_model4.predict(dangjin_test)

In [ ]:
dangjin_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold4.h5")

In [ ]:
dangjin0 = dangjin_model0.predict(dangjin_test)
dangjin1 = dangjin_model1.predict(dangjin_test)
dangjin2 = dangjin_model2.predict(dangjin_test)
dangjin3 = dangjin_model3.predict(dangjin_test)
dangjin4 = dangjin_model4.predict(dangjin_test)

In [ ]:
ulsan_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold4.h5")

In [ ]:
dangjin_test.shape

(24, 72, 14, 3)

In [ ]:
ulsan_test.shape

(24, 72, 14, 3)

In [ ]:
ulsan0 = ulsan_model0.predict(ulsan_test)
ulsan1 = ulsan_model1.predict(ulsan_test)
ulsan2 = ulsan_model2.predict(ulsan_test)
ulsan3 = ulsan_model3.predict(ulsan_test)
ulsan4 = ulsan_model4.predict(ulsan_test)

In [ ]:

floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4 ) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5


In [ ]:
'''
floating = (floating0 + floating1 + floating2) / 3
warehouse = (warehouse0 + warehouse1 + warehouse2 ) / 3
dangjin = (dangjin0 + dangjin1 + dangjin2) / 3
ulsan = (ulsan0 + ulsan1 + ulsan2 ) / 3
'''

'\nfloating = (floating0 + floating1 + floating2) / 3\nwarehouse = (warehouse0 + warehouse1 + warehouse2 ) / 3\ndangjin = (dangjin0 + dangjin1 + dangjin2) / 3\nulsan = (ulsan0 + ulsan1 + ulsan2 ) / 3\n'

In [ ]:
#sample_submission2 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/sample_submission.csv")
sample_submission = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/neural_prophet_validation.csv")

In [ ]:
#sample_submission2

In [ ]:
sample_submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.0,0.000000,0.0,0.000000
1,2021-02-01 02:00:00,0.0,0.000000,0.0,0.000000
2,2021-02-01 03:00:00,0.0,0.000000,0.0,0.000000
3,2021-02-01 04:00:00,0.0,0.000000,0.0,0.000000
4,2021-02-01 05:00:00,0.0,0.000000,0.0,0.651203
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.0,1.125614,0.0,2.495743
1388,2021-07-08 21:00:00,0.0,1.879234,0.0,3.404133
1389,2021-07-08 22:00:00,0.0,0.933403,0.0,4.150723
1390,2021-07-08 23:00:00,0.0,0.127186,0.0,3.220158


* 시간은 내일 00시부터 23시까지 !! (가져온 날씨 데이터가 그럼)

In [ ]:
#loc는 끝 인덱스도 포함....
#sample_submission.loc[815:839] #6/14일 제출
#sample_submission.loc[839:862] #6/15일
#sample_submission.loc[863:886] #6/16일
#sample_submission.loc[887:910] #6/17일
#sample_submission.loc[911:934] #6/18일
#sample_submission.loc[935:958] #6/19일
#sample_submission.loc[959:982] #6/20일
#sample_submission.loc[983:1006] #6/20일
#sample_submission.loc[1295:1318] #7/4
sample_submission.loc[1319:1342]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1319,2021-07-05 24:00:00,1.796295,4.897259,0.000000,4.635559
1320,2021-07-06 01:00:00,0.974709,5.701443,0.000000,5.627445
1321,2021-07-06 02:00:00,0.000000,4.973846,0.000000,5.760998
1322,2021-07-06 03:00:00,0.000000,5.386650,0.000000,5.329369
1323,2021-07-06 04:00:00,0.681019,6.659737,0.000000,6.699356
1324,2021-07-06 05:00:00,0.000000,5.333271,0.000000,8.259907
1325,2021-07-06 06:00:00,0.000000,2.601074,0.000000,7.156857
1326,2021-07-06 07:00:00,1.045525,8.011665,0.000000,10.943401
1327,2021-07-06 08:00:00,29.679703,35.029007,31.673172,35.992615
1328,2021-07-06 09:00:00,96.714920,92.263351,118.742386,87.238235


In [ ]:
floating

array([[  2.4601412],
       [  2.5823867],
       [  2.5504615],
       [  2.43695  ],
       [  2.444633 ],
       [  2.4066174],
       [  2.8081899],
       [  5.6439066],
       [ 20.768415 ],
       [ 94.70459  ],
       [198.16928  ],
       [270.14468  ],
       [327.27716  ],
       [312.74536  ],
       [273.16754  ],
       [229.05562  ],
       [167.22116  ],
       [110.38139  ],
       [ 63.569733 ],
       [ 14.143152 ],
       [  2.0237184],
       [  1.4589626],
       [  1.5775397],
       [  1.883469 ]], dtype=float32)

In [ ]:
warehouse

array([[  1.8233902],
       [  1.7961569],
       [  1.9179659],
       [  2.1867037],
       [  2.3338091],
       [  2.3344638],
       [  2.3360944],
       [  2.5504143],
       [  7.017029 ],
       [ 32.971817 ],
       [ 77.30202  ],
       [113.95321  ],
       [148.9844   ],
       [146.87657  ],
       [128.82712  ],
       [106.25604  ],
       [ 81.41909  ],
       [ 48.654377 ],
       [ 19.418142 ],
       [  2.9357414],
       [  1.699897 ],
       [  1.7070005],
       [  1.7396553],
       [  1.9013088]], dtype=float32)

In [ ]:
floating.shape

(24, 1)

In [ ]:
sample_submission.loc[1319:1342].shape

(24, 5)

In [ ]:
sample_submission.loc[1319:1342, 'dangjin_floating'] = floating
sample_submission.loc[1319:1342, 'dangjin_warehouse'] = warehouse
sample_submission.loc[1319:1342, 'dangjin'] = dangjin
sample_submission.loc[1319:1342, 'ulsan'] = ulsan

In [ ]:
sample_submission.loc[1319:1342]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1319,2021-07-05 24:00:00,1.624381,1.540654,6.502242,126.312035
1320,2021-07-06 01:00:00,1.543915,1.630076,5.913037,126.312035
1321,2021-07-06 02:00:00,1.407984,1.648800,5.377753,126.312035
1322,2021-07-06 03:00:00,1.365938,1.739118,4.929554,126.312035
1323,2021-07-06 04:00:00,1.410169,1.802291,4.559999,126.312035
1324,2021-07-06 05:00:00,1.459503,1.828981,4.270782,126.312035
1325,2021-07-06 06:00:00,1.704308,1.930914,4.647805,126.312035
1326,2021-07-06 07:00:00,2.320069,2.246169,5.061444,126.312035
1327,2021-07-06 08:00:00,3.621434,3.078272,6.995732,126.312035
1328,2021-07-06 09:00:00,20.301931,8.041381,24.431065,126.312035


In [ ]:
sample_submission.to_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0705_submission.csv", index = False)

In [ ]:
ls